In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# ----------------------------------------
# 1) Generate tiny synthetic dataset
# ----------------------------------------
# Two classes in 2D:
#  - class 0 around (-1, -1)
#  - class 1 around (+1, +1)

num_samples = 20

class0 = torch.randn(num_samples, 2) * 0.3 + torch.tensor([-1.0, -1.0])
class1 = torch.randn(num_samples, 2) * 0.3 + torch.tensor([1.0, 1.0])

X = torch.cat([class0, class1], dim=0)
y = torch.cat([torch.zeros(num_samples), torch.ones(num_samples)]).long()

# ----------------------------------------
# 2) Define a simple neural network
# ----------------------------------------
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 2)    # 2 output classes
        )

    def forward(self, x):
        return self.net(x)

model = SimpleClassifier()

# ----------------------------------------
# 3) Training setup
# ----------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ----------------------------------------
# 4) Training loop
# ----------------------------------------
epochs = 200

for epoch in range(epochs):
    optimizer.zero_grad()

    logits = model(X)
    loss = criterion(logits, y)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 40 == 0:
        pred = torch.argmax(logits, dim=1)
        acc = (pred == y).float().mean().item()
        print(f"Epoch {epoch+1:3d} | Loss = {loss.item():.4f} | Acc = {acc*100:.1f}%")

# ----------------------------------------
# 5) Test on new samples
# ----------------------------------------
test_points = torch.tensor([
    [-1.2, -0.8],
    [1.1,  0.9],
    [0.0,  0.0]
])

with torch.no_grad():
    out = model(test_points)
    preds = torch.argmax(out, dim=1)
    print("\nTest predictions:", preds.tolist())


Epoch  40 | Loss = 0.0070 | Acc = 100.0%
Epoch  80 | Loss = 0.0018 | Acc = 100.0%
Epoch 120 | Loss = 0.0011 | Acc = 100.0%
Epoch 160 | Loss = 0.0007 | Acc = 100.0%
Epoch 200 | Loss = 0.0005 | Acc = 100.0%

Test predictions: [0, 1, 1]


In [49]:
# Prepare input examples for model logging
# Use a small sample of test data as input example
input_example = np.array([
    [-0.66, -1.77],
    [-0.74, -1.42],
    [-1.50, -0.92]
], dtype=np.float32)

# Convert to torch.float32 tensor
input_example_tensor = torch.from_numpy(input_example).float()

# Convert to numpy array for MLflow
print(f"Input example shape: {input_example.shape}")
print(f"Input example (first sample): {input_example[0]}")

# Optionally, we can also create a model signature manually
import mlflow.types.schema as schema
from mlflow.models.signature import infer_signature

# Create model prediction for signature inference
model.eval()
with torch.no_grad():
    example_output = model(X).numpy()

# Infer signature from input and output
signature = infer_signature(input_example, example_output)
print(f"Model signature: {signature}")
print("✓ Input examples and signature prepared for MLflow logging")

2025/11/25 19:02:09 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
2025/11/25 19:02:09 WARNING mlflow.models.signature: Failed to infer schema for outputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. To see the full traceback, set logging level to DEBUG.


Input example shape: (3, 2)
Input example (first sample): [-0.66 -1.77]
Model signature: inputs: 
  [Any (required)]
outputs: 
  [Any (required)]
params: 
  None

✓ Input examples and signature prepared for MLflow logging


In [50]:
import onnx
import onnxruntime as ort

def convert_to_onnx(model, input_size=(1,2), onnx_path="onnx_model.onnx"):
    """Convert PyTorch model to ONNX format"""
    model.eval()
    
    # Create dummy input for tracing
    dummy_input = torch.randn(input_size, dtype=torch.float32)
    
    # Export to ONNX
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    
    print(f"Model exported to ONNX format: {onnx_path}")
    return onnx_path

def verify_onnx_model(onnx_path, test_data):
    """Verify ONNX model works correctly"""
    # Load ONNX model
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    
    # Create ONNX Runtime session
    ort_session = ort.InferenceSession(onnx_path)
    
    # Test with a small batch
    test_input = test_data[:5].numpy()  # Take first 5 samples
    ort_inputs = {ort_session.get_inputs()[0].name: test_input}
    ort_outputs = ort_session.run(None, ort_inputs)
    
    print(f"ONNX model verification successful. Output shape: {ort_outputs[0].shape}")
    return True

# Convert and verify ONNX model
onnx_path = "mymodel.onnx"
convert_to_onnx(model, input_size=(1, 2), onnx_path=onnx_path)
verify_onnx_model(onnx_path, X)

/tmp/ipykernel_103/739978797.py:12: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1125 19:02:11.476000 103 .local/lib/python3.10/site-packages/torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W1125 19:02:11.964000 103 .local/lib/python3.10/site-packages/torch/onnx/_internal/exporter/_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `SimpleClassifier([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SimpleClassifier([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Model exported to ONNX format: mymodel.onnx
ONNX model verification successful. Output shape: (5, 2)


True

In [51]:
import mlflow
import onnxmltools
from onnxmltools.convert.common.data_types import FloatTensorType

REGISTERED_MODEL_NAME_ONNX = "onnx_classifier_notebook"
with mlflow.start_run() as run:
    # Log the PyTorch model to MLflow with input example and signature
    print("Logging PyTorch model to MLflow...")
    mlflow.pytorch.log_model(
        model, 
        "classifier_pytorch",
        registered_model_name="pytorch_classifier_notebook",
        input_example=input_example
    )
    
    # Log the ONNX model to MLflow with input example and signature  
    print("Logging ONNX model to MLflow...")
    onnx_model = onnx.load(onnx_path)
    mlflow.onnx.log_model(
        onnx_model,
        "classifier_onnx",
        registered_model_name=f"{REGISTERED_MODEL_NAME_ONNX}",
        input_example=input_example
    )
    
    print("Models logged successfully to MLflow!")
    print(f"MLflow Run ID: {run.info.run_id}")
    print("✓ Models logged with input examples and signatures!")

2025/11/25 19:02:14 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: ModuleNotFoundError("No module named 'scipy'"). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`.


Logging PyTorch model to MLflow...


2025/11/25 19:02:24 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      -0.6600000262260437,
      -1.7699999809265137
    ],
    [
      -0.7400000095367432,
      -1.4199999570846558
    ],
    [
      -1.5,
      -0.9200000166893005
    ]
  ]
}. Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: mat1 and mat2 must have the same dtype, but got Double and Float
Successfully registered model 'pytorch_classifier_notebook'.
2025/11/25 19:02:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: pytorch_classifier_no

Logging ONNX model to MLflow...


Successfully registered model 'onnx_classifier_notebook'.
2025/11/25 19:02:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: onnx_classifier_notebook, version 7


Models logged successfully to MLflow!
MLflow Run ID: pl1o-3eod-7sl2-88uj
✓ Models logged with input examples and signatures!


Created version '7' of model 'onnx_classifier_notebook'.
